In [1]:
import random

from process import Processor, Buffer

In [2]:
TIME_LIMIT = 100000
LAMBDA = 0.1
EXPECTED_WORK_TIME = 40

In [3]:
p1 = Processor('P1', EXPECTED_WORK_TIME, Buffer(4))
p2 = Processor('P2', EXPECTED_WORK_TIME)

In [4]:
def get_requests_timings(lam):
    timings = []
    current_time = 0
    while current_time < TIME_LIMIT:
        next_request_in = int(random.expovariate(lam))
        timings.append(current_time + next_request_in)
        current_time += next_request_in
        
    return timings

# TODO: tests for this
def count_intervals(timings):
    ''' Returns intervals between events in timings. '''
    intervals = list(map(lambda i: 
        rejected_requests[i] - rejected_requests[i-1], 
        reversed(range(1, len(rejected_requests)))))
    return intervals

In [5]:
# THIS FUNCTION IS NOT TESTED!!!

def simulate(time_limit, requests, thresholds, processors):
    ''' Simulates work of `processes` in discrete time till `time_limit`.
    
    Note:
        thresholds are CDF values. So to "encode" 2 processors with 
        prob of 1/3 to get in the 1st and 2/3 in the second use 
        the following array: [0.33, 1]
    
    Args:
        time_limit(int): amount of clocks that should be simulated
        thresholds(list of floats in [0;1)): probability thresholds, that
            certain request would get into i-th processor
        processors(list of Processor`s): processes in system
        
    Returns:
        results(Results): report about performed simulation
    '''
    # requests = get_requests_timings(LAMBDA)
    rejected_requests = []
    processed_requests = []
    working_time = 0
    
    procs_stats = {p.name: {
        'queue_len': [],
        'lost': 0,
        'done': 0,
        'processing_time': [],
        'lose_prob': 0.0,
        'performance': 0,
        } for p in processors}

    for i in range(time_limit):
        if i % 5000 == 0:
            print(i, end=' => ')
            
        if i in requests:
            # define in which arm request should be processed
            arm_check = random.random()
            # check if the arm can process request (either processor or buffer)
            attachment_result = 0
            attached_to = ''
            for proc_index, thresh in enumerate(thresholds):
                if arm_check < thresh:
                    attachment_result = processors[proc_index].add_process(i)
                    attached_to = processors[proc_index].name
                    procs_stats[attached_to]['queue_len'].append(
                        processors[proc_index].get_queue_len()
                    )
                    break

            if attachment_result == 0:
                # the processor was not able to take request
                rejected_requests.append(i)
                # add one to the counter of *rejected* processes for
                # this responsible processor
                procs_stats[attached_to]['lost'] += 1
            else:
                # add one to the counter of *performed* processes for
                # this responsible processor
                processed_requests.append(i+attachment_result)
                procs_stats[attached_to]['done'] += 1
                procs_stats[attached_to]['processing_time'].append(attachment_result)
            
        for p in processors:
            p.process()
            if p.is_busy():
                working_time += 1
                
    if len(processed_requests) + len(rejected_requests) != len(requests) - 1:
        print("you fooled me!")
                
    # calculate results
    for p in procs_stats:
        # for all processors, probability of losing request =
        # = #(lost requests) / #(all request to this processor)
        procs_stats[p]['lose_prob'] = procs_stats[p]['lost'] / (procs_stats[p]['done'] + procs_stats[p]['lost'])
        
    load = (len(requests)/sum(requests)) / (len(processed_requests)/(sum(processed_requests)))
    print('Work load: ', load)
    
    utilization = working_time / (time_limit*len(processors))
    print('Utilization: ', utilization)
        
    return (procs_stats, rejected_requests)

In [6]:
class Results:
    def __init__(self):
        self.load = 0.0           # нагрузка
        self.utilization = 0.0   # загрузка
        self.queue_len = {}
        self.n_requests = {}
        self.idle_time = {}
        self.processing_time = {}
        self.lose_prob = {}
        self.performance = {}

In [7]:
requests = get_requests_timings(LAMBDA)
print(requests[:10])
rejected_requests = []

for i in range(TIME_LIMIT):
    if i % 5000 == 0:
        print(i, end=' => ')
    p1.process()
    p2.process()
    if i in requests:
        # define in which arm request should be processed
        arm_check = random.random()
        # check if the arm can process request (either processor or buffer)
        attachment_result = 0
        if arm_check < 0.33:
            attachment_result = p1.add_process(i)
        else:
            attachment_result = p2.add_process(i)
        
        if attachment_result == 0:
            # the processor was not able to take request
            rejected_requests.append(i)

[4, 10, 11, 70, 82, 85, 85, 101, 107, 154]
0 => 5000 => 10000 => 15000 => 20000 => 25000 => 30000 => 35000 => 40000 => 45000 => 50000 => 55000 => 60000 => 65000 => 70000 => 75000 => 80000 => 85000 => 90000 => 95000 => 

In [8]:
intervals = count_intervals(rejected_requests)
print("Workload of System1: ", 1/(sum(intervals)/len(intervals)))

Workload of System1:  0.055453216667000385


### Система 2

In [9]:
p1 = Processor('P1', EXPECTED_WORK_TIME, Buffer(1))
p2 = Processor('P2', EXPECTED_WORK_TIME)
p3 = Processor('P3', EXPECTED_WORK_TIME)

In [10]:
#requests = get_requests_timings(LAMBDA)
# print(requests[:10])
rejected_requests = []

for i in range(TIME_LIMIT):
    if i % 5000 == 0:
        print(i, end=' => ')
    p1.process()
    p2.process()
    p3.process()
    if i in requests:
        # define in which arm request should be processed
        arm_check = random.random()
        # check if the arm can process request (either processor or buffer)
        attachment_result = 0
        if arm_check < 0.33:
            attachment_result = p1.add_process(i)
        elif arm_check < .66:
            attachment_result = p2.add_process(i)
        else:
            attachment_result = p3.add_process(i)
        
        if attachment_result == 0:
            # the processor was not able to take request
            rejected_requests.append(i)

0 => 5000 => 10000 => 15000 => 20000 => 25000 => 30000 => 35000 => 40000 => 45000 => 50000 => 55000 => 60000 => 65000 => 70000 => 75000 => 80000 => 85000 => 90000 => 95000 => 

In [11]:
intervals = count_intervals(rejected_requests)
print("Workload of System2: ", 1/(sum(intervals)/len(intervals)))

Workload of System2:  0.04884648533803692


In [12]:
stats, rejected = simulate(TIME_LIMIT, requests, [1/3, 2/3, 1], [p1, p2, p3])

0 => 5000 => 10000 => 15000 => 20000 => 25000 => 30000 => 35000 => 40000 => 45000 => 50000 => 55000 => 60000 => 65000 => 70000 => 75000 => 80000 => 85000 => 90000 => 95000 => you fooled me!
Work load:  0.9999112627931045
Utilization:  0.60843


In [13]:
print(stats)

{'P1': {'processing_time': [32, 22, 46, 102, 5, 4, 3, 53, 17, 44, 7, 29, 26, 25, 23, 14, 50, 2, 8, 37, 117, 21, 6, 13, 45, 29, 197, 15, 97, 4, 59, 3, 24, 5, 16, 42, 12, 15, 6, 51, 7, 8, 6, 95, 20, 29, 31, 16, 28, 44, 35, 52, 71, 2, 11, 15, 1, 69, 10, 75, 235, 3, 25, 23, 76, 2, 62, 29, 24, 1, 18, 26, 3, 21, 19, 6, 14, 26, 18, 2, 23, 70, 6, 14, 11, 35, 128, 75, 13, 80, 91, 38, 8, 7, 8, 20, 26, 59, 73, 22, 30, 1, 29, 181, 13, 7, 15, 26, 7, 101, 79, 94, 53, 12, 19, 74, 18, 34, 43, 10, 48, 18, 2, 21, 2, 3, 22, 23, 39, 14, 15, 7, 10, 38, 64, 8, 43, 5, 4, 115, 11, 100, 49, 79, 39, 9, 74, 37, 18, 8, 4, 56, 55, 10, 33, 48, 19, 6, 27, 24, 33, 75, 21, 56, 134, 127, 23, 39, 17, 17, 30, 40, 12, 61, 44, 13, 357, 26, 36, 3, 43, 240, 38, 8, 45, 80, 68, 8, 56, 18, 11, 33, 10, 38, 13, 13, 46, 148, 24, 44, 22, 30, 45, 41, 23, 25, 12, 46, 55, 17, 4, 13, 15, 53, 8, 101, 70, 130, 86, 21, 47, 80, 8, 8, 4, 10, 22, 95, 21, 74, 80, 70, 41, 27, 14, 18, 124, 65, 37, 111, 17, 24, 74, 104, 34, 67, 65, 10, 142, 100,

In [14]:
print(max(requests))

100015
